# Graficacion Meteorologica Mediante Interpolaciones 🌡

<img src=Imagenes/App2.jpg width=300px>
<hr>

<h2>Tomamos los datos consumiendo una api del clima</h2>
<h5>https://developer.weatherunlocked.com/documentation/localweather/forecast</h5>
<h3>Elegimos la locacion de Turquia estambul</h3>
<img src=Imagenes/MemeTurco.jpg width=300px>
<hr>

## Consumo De Api para la Generacion de datos (1 Semana)

In [4]:
#Importamos las librerias necesarias
using HTTP
using JSON

#Pido los datos desde la api
url = "http://api.weatherunlocked.com/api/forecast/41.01384,-28.94966?app_id=f8341b8f&app_key=380d876792d6ac1b32c5258c07c010a4"
res = HTTP.get(url)
json = JSON.parse(String(res.body))

Temperatura_max = []
Temperatura_min = []
temperatura = []
x = json["Days"]

for i in x
    push!(Temperatura_max, i["temp_max_c"])
    push!(Temperatura_min, i["temp_min_c"])
end

for i in 1:1:length(Temperatura_max)
    temp_max = Temperatura_max[i]
    temp_min = Temperatura_min[i]
    prom = temp_max + temp_min
    push!(temperatura, round(prom/2 , digits=2))
end
print(temperatura)

Any[20.65, 22.5, 22.35, 22.45, 23.7, 22.6, 20.2]

## Datos Meteorologicos anuales de Estambul

In [4]:
api_key = "790e4819ffc841c5b1810355241106"
location = "Istanbul"
nombre_doc = location * "_weather_data.json"

data_list = []

for month in 1:12
    start_date = "2023-$(lpad(month, 2, '0'))-01"
    end_date = month != 12 ? "2023-$(lpad(month, 2, '0'))-31" : "2023-12-31"

    url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q=$(location)&tp=24&date=$(start_date)&enddate=$(end_date)&format=json&key=$(api_key)"

    response = HTTP.get(url)

    if response.status == 200
        data = JSON.parse(String(response.body))
        push!(data_list, data)
        println("Datos del mes $month obtenidos correctamente")
    else
        println("Error al obtener datos del mes $month: $(response.status)")
    end
end

months = [
    "Enero",
    "Febrero",
    "Marzo",
    "Abril",
    "Mayo",
    "Junio",
    "Julio",
    "Agosto",
    "Septiembre",
    "Octubre",
    "Noviembre",
    "Diciembre",
]

max_temps = []
min_temps = []
avg_temps = []

for month_data in data_list
    month_max_temps = []
    month_min_temps = []
    month_avg_temps = []

    for day in month_data["data"]["weather"]
        max_temp = day["maxtempC"]
        min_temp = day["mintempC"]
        avg_temp = day["avgtempC"]

        push!(month_max_temps, max_temp)
        push!(month_min_temps, min_temp)
        push!(month_avg_temps, avg_temp)
    end

    push!(max_temps, month_max_temps)
    push!(min_temps, month_min_temps)
    push!(avg_temps, month_avg_temps)
end



Datos del mes 1 obtenidos correctamente
Datos del mes 2 obtenidos correctamente
Datos del mes 3 obtenidos correctamente
Datos del mes 4 obtenidos correctamente
Datos del mes 5 obtenidos correctamente
Datos del mes 6 obtenidos correctamente
Datos del mes 7 obtenidos correctamente
Datos del mes 8 obtenidos correctamente
Datos del mes 9 obtenidos correctamente
Datos del mes 10 obtenidos correctamente
Datos del mes 11 obtenidos correctamente
Datos del mes 12 obtenidos correctamente


<hr>

## Interpolacion de Taylor

In [ ]:
using SymPy
using Plots




